In [2]:
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

## What does this data represent and what are we trying to find out about it?
several rows and 13 collumns.  Each collumn represents a different numerical statistic such as accessability to highways and per capita crime. Each row is a different sample of homes.  
The targets are the median values of homes in thousands of dollars.  Remember that this data was gathered in the 70's.


## preparing the data
Since we have 13 categories that can very widely, we need to do featurewise normalization.

In [6]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)

test_data -= mean
test_data /= std


In [7]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                          input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [9]:
build_model()

## evaluating our model using k-fold cross-validation
Since we have a relativly small training set, we will get a lot of variation depending on which validation set we select.  Because of this, we cannot just select 100 or so data points and use them in our validation set.  
Instead, we will need to use *k-fold cross-validation*.  K-fold cross-validation is when we partition the training data into k groups.  We instantiate K identical models and split use one of our k partitions for each as a validation set.  This takes a lot longer but gives us better results than just creating a validation set.

In [ ]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print(f'processing fold #{i}')
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
        train_targets[(i+1) * num_val_samples:]],
        axis=0
        )
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
            epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_targ)